# LEMKE- HOWSON
[Assignment 1] We focus on perturbations over payoffs. You are required to study how the equilibria of a game vary once some payoff of the matrix is perturbed. For instance, given the following game

(1,1) (0,0)

(0,0) (0,0)

a perturbation can be:

(1,1) (0,0)

(0,0) (e,e)

where e > 0 is arbitrarily close to zero. You will be required to understand how the best-response regions vary as perturbation e varies (in particular, studying the regions and their labels used by the Lemke-Howson algorithm) and how the set of equilibria changes (e.g., new equilibria may appear, some equilibria may collapse to a single equilibrium, some equilibria may disappear). 

In [1]:
from amplpy import AMPL
import pandas as pd

In [2]:
# Initialize AMPL
ampl = AMPL()

# Nash equilibrium function
def nash_equilibrium(players, actions, **kwargs):
    # AMPl for data
    I = actions
    P = players

    U = {player: kwargs['U' + str(player)] for player in P}
    
    # AMPL for model
    ampl.eval(r"""
        set P; # Players
        set I; # Actions
        
        param U1 {I, I};
        param U2 {I, I};
        param M{P};
        
        var s1 {I} >= 0;
        var s2 {I} >= 0;
        var b1 {I} binary;
        var b2 {I} binary;
        var v{P};
        
        subject to cons1_1{i in I}: s1[i] - b1[i] <= 0;
        subject to cons1_2{i in I}: s2[i] - b2[i] <= 0;
        
        subject to cons2_1{i in I}: v[1] - sum{j in I} (U1[i,j] * s2[j]) - M[1]*(1-b1[i]) <= 0;
        subject to cons2_2{j in I}: v[2] - sum{i in I} (U2[i,j] * s1[i]) - M[2]*(1-b2[j]) <= 0;
        
        subject to cons3_1{i in I}: v[1] - sum{j in I} (U1[i,j] * s2[j]) >= 0;
        subject to cons3_2{j in I}: v[2] - sum{i in I} (U2[i,j] * s1[i]) >= 0;
        
        subject to sumToOne_1: sum{i in I} s1[i] == 1;
        subject to sumToOne_2: sum{j in I} s2[j] == 1;
        
        maximize obj: v[1] + v[2];
    """)

    ampl.set['P'] = P
    ampl.set['I'] = I

    U_df = {player: pd.DataFrame(U[player], index=I, columns=I) for player in P}
    
    for player in P:
        ampl.param['U' + str(player)] = U_df[player]

    ampl.param['M'] = {player: 1000 for player in P}

    # AMPL solver
    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    ampl.display('v', 's1', 's2')

## two actions game

In [3]:


I = ["a1", "a2", "a3"]
P = [1, 2]


U1 = [[2,1,0], 
    [1,2,0],
    [0,0,0]]

U2 = [[1,2,0], 
    [2,1,0],
    [0,0,0]]

nash_equilibrium(players=P, actions=I, U1 = U1, U2 = U2)

Gurobi 9.5.1: optimal solution; objective 3
16 simplex iterations
1 branch-and-cut nodes
plus 8 simplex iterations for intbasis
:     v    s1    s2     :=
1    1.5    .     .
2    1.5    .     .
a1    .    0.5   0.5
a2    .    0.5   0.5
a3    .    0     0
;

